In [1]:
from gensim.models import Word2Vec, KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [18]:
!pip install rank_bm25
!git clone https://github.com/cr-nlp/project1-2023.git

import urllib.request as re
from collections import defaultdict
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')

def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename,encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename,encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename,encoding='utf-8') as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc


def text2TokenList(text):
    stopword = stopwords.words('english')
    word_tokens = word_tokenize(text.lower())
    return [word for word in word_tokens if word not in stopword and len(word) > 2]

def train_word2vec_model(corpus):
    # Train Word2Vec model
    model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)
    return model

def document_vector(doc, model):
    # Create a document vector
    return np.mean([model.wv[word] for word in doc if word in model.wv.key_to_index], axis=0)

def run_word2vec_model():
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    # Tokenize and prepare corpus for Word2Vec training
    corpus = [text2TokenList(dicDoc[doc]) for doc in dicDoc]
    word2vec_model = train_word2vec_model(corpus)

    # Generate embeddings for documents and queries
    doc_embeddings = {doc: document_vector(text2TokenList(dicDoc[doc]), word2vec_model) for doc in dicDoc}
    query_embeddings = {req: document_vector(text2TokenList(dicReq[req]), word2vec_model) for req in dicReq}

    # Compute cosine similarities
    doc_matrix = np.array(list(doc_embeddings.values()))
    query_matrix = np.array(list(query_embeddings.values()))
    cosine_sim_matrix = cosine_similarity(query_matrix, doc_matrix)

    # Evaluate the model
    from sklearn.metrics import ndcg_score 
    ndcg_values = []
    for i, req in enumerate(dicReq):
        true_relevance = np.array([dicReqDoc[req].get(doc, 0) for doc in dicDoc])
        predicted_relevance = cosine_sim_matrix[i]
        ndcg_value = ndcg_score([true_relevance], [predicted_relevance])
        ndcg_values.append(ndcg_value)

    average_ndcg = np.mean(ndcg_values)
    print("Word2Vec Average NDCG:", average_ndcg)

# Run the Word2Vec model
run_word2vec_model()  


fatal: destination path 'project1-2023' already exists and is not an empty directory.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rosel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rosel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Word2Vec Average NDCG: 0.30354577459717424
